Nos conectamos al drive, donde esta nuestro dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Tratamiento de las imagenes para las CNN**

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from collections import defaultdict
import time

Recuperamos la direccion del dataset y el haarcascade

In [3]:
dataset_path = "/content/drive/My Drive/valVGGface2"
face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/haarcascade_frontalface_default.xml')

Parametros que utilizaremos para el tratamiento de datos

In [4]:
IMG_SIZE = (224, 224) #dimension de la redimension
#listas para almacenar imagenes(X), etiquetas(Y) y clases(classes)
X = []
Y = []
classes = []
image_count_per_class = defaultdict(int)

Procesaremos las imagenes de cada persona y llenaremos X, Y y classes con datos correspondientes

In [5]:
#detectamos y procesamos cada imagen
for person_name in os.listdir(dataset_path): #recuperamos el nombre de la persona, vendra siendo el nombre de la carpeta
    person_path = os.path.join(dataset_path, person_name) #recuperamos la direccion de la carpeta de la persona
    if os.path.isdir(person_path): #verifica que existe la carpeta
        classes.append(person_name) #agregamos el nombre de las personas a las clases
        for img_name in os.listdir(person_path): #recuperamos cada imagen de la carpeta de la persona
            img_path = os.path.join(person_path, img_name) #obtenemos la direccion de la imagen de cada persona
            img = cv2.imread(img_path) #obtenemos la imagen
            if img is not None: #verifica que la imagen NO es None
                image_count_per_class[person_name] += 1
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #pasamos la imagen a escala de grises
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5) #obtenemos la cara de la persona que esta en la imagen
                for (x, y, w, h) in faces:
                    cropped_face = gray[y:y+h, x:x+w] #usamos la imagen en escala de grises
                    resized_face = cv2.resize(cropped_face, IMG_SIZE) #redimensionamos a 224x224
                    X.append(resized_face / 255.0)  #normalizamos entre 0-1
                    Y.append(classes.index(person_name))

Pasamos X y Y a arrays de numpy

In [6]:
X = np.array(X)
Y = np.array(Y)

Total de imagenes

In [7]:
print(f"Total de imágenes en X: {len(X)}")

Total de imágenes en X: 3720


Total de imagenes por persona

In [8]:
for person_name, count in image_count_per_class.items():
    print(f"Imágenes para {person_name}: {count}")

Imágenes para n000148: 367
Imágenes para n000029: 454
Imágenes para n000129: 585
Imágenes para n000078: 458
Imágenes para n000001: 424
Imágenes para n000040: 180
Imágenes para n000106: 279
Imágenes para n000082: 441
Imágenes para n000009: 136
Imágenes para n000149: 522
Imágenes para n000178: 208
Imágenes para guido: 400


Redimensionamos las imagenes obtenidas(X) para que tengan un solo canal

In [9]:
#redimensionamos para que las imagenes tengan 1 canal
X = X.reshape(X.shape[0], IMG_SIZE[0], IMG_SIZE[1], 1)

Convertimos las etiquetas a formas categoricas

In [10]:
#convertimos las etiquetas a formato categorico (one-hot encoding)
Y = to_categorical(Y, num_classes=len(classes))

Ahora dividimos los datos que tenemos, en entrenamiento y validacion

In [11]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [12]:
#imprimimos las dimensiones de los conjuntos de datos
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {Y_val.shape}")

X_train shape: (2976, 224, 224, 1)
y_train shape: (2976, 12)
X_val shape: (744, 224, 224, 1)
y_val shape: (744, 12)


# **Creacion del modelo**

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

Parametros que utilizaremos durante la creacion del modelo

In [14]:
#parametros del modelo
input_shape = (224, 224, 1)  #imagenes en escala de grises (224x224x1)
num_classes = len(classes)  #numero de clases en classes

Creamos el modelo CNN

In [15]:
modelo = Sequential([
    #primera capa convolucional
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    #primera capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #segunda capa convolucional
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    #segunda capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #tercera capa convolucional
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    #tercera capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #cuarta capa convolucional
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    #cuarta capa maxpooling
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    #Aplanado (Flatten)
    Flatten(),

    #primera capa completamente conectada (256 unidades)
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    #segunda capa completamente conectada (200 unidades)
    Dense(200, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    #tercera capa completamente conectada (150 unidades)
    Dense(150, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    #cuarta capa completamente conectada (100 unidades)
    Dense(100, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    #capa de salida (Softmax para clasificacion)
    Dense(num_classes, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Usaremos una Loss Function modifica para agregar la penalizacion

In [16]:
def Categorical_Cross_Entropy_with_Penalization(y_true, y_pred):
    #Entropia cruzada estandar
    perdida_base = K.categorical_crossentropy(y_true, y_pred)

    #Calculo de la entropia de las predicciones (medida de incertidumbre)
    entropia_pred = -K.sum(y_pred * K.log(y_pred + K.epsilon()), axis=-1)

    #Penalizacion adicional (puedes ajustar el factor segun tu necesidad)
    penalizacion = 0.1 * entropia_pred  # Factor de peso para la penalizacion

    #Combinar perdida base y penalizacion
    return perdida_base + penalizacion


Compilamos el modelo

In [17]:
modelo.compile(
    optimizer='adam',
    loss=Categorical_Cross_Entropy_with_Penalization,
    metrics=['accuracy']
)

Veamos un resumen de nuestro modelo

In [18]:
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 24, 24, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36864)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       9,437,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 9,927,886 (37.87 MB)

 Trainable params: 9,925,514 (37.86 MB)

 Non-trainable params: 2,372 (9.27 KB)

Criterio de parada

In [19]:
#Configuracion de Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',       #Monitorea la perdida en el conjunto de validacion
    patience=10,               #Detiene si no hay mejora tras 10 epocas consecutivas
    restore_best_weights=True #Restaura los pesos del mejor modelo
)

Entrenamos nuestro modelo

In [20]:
start_time = time.time()
history = modelo.fit(
    X_train, Y_train,  #datos de entrenamiento y etiquetas
    validation_data=(X_val, Y_val),  #datos de validacion y etiquetas
    epochs=100,  #numero de epocas
    batch_size=32,  #tamano del lote
    verbose=1,
    callbacks=[early_stopping]
)
end_time = time.time()

Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - accuracy: 0.1167 - loss: 3.6137 - val_accuracy: 0.0995 - val_loss: 10.3431
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.2319 - loss: 2.7358 - val_accuracy: 0.0995 - val_loss: 13.9541
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.3537 - loss: 2.1904 - val_accuracy: 0.0995 - val_loss: 11.3104
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.3947 - loss: 1.9785 - val_accuracy: 0.1169 - val_loss: 8.8595
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.4689 - loss: 1.7284 - val_accuracy: 0.1935 - val_loss: 3.0669
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.4890 - loss: 1.6935 - val_accuracy: 0.3481 - val_loss: 2.1161
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - accuracy: 0.5902 - loss: 1.3658 - val_accuracy: 0.2782 - val_loss: 2.6236
Epoch 8/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.6290 - loss: 1.2152 - val_accura

Veamos cuanto duro el entrenamiento

In [21]:
training_time = end_time - start_time
minutes, seconds = divmod(training_time, 60)
print(f"El entrenamiento tomó {int(minutes)} minutos y {seconds:.2f} segundos.")


El entrenamiento tomó 4 minutos y 56.07 segundos.


Veamos el Loss y el Accuracy del modelo entrenado

In [22]:
loss, accuracy = modelo.evaluate(X_val, Y_val)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8515 - loss: 0.5201
Loss: 0.4905, Accuracy: 0.8642


Guardemos el modelo en un archivo .keras

In [23]:
modelo.save('reconocimientoFacialModelV10.keras')

In [24]:
classes

['n000148',
 'n000029',
 'n000129',
 'n000078',
 'n000001',
 'n000040',
 'n000106',
 'n000082',
 'n000009',
 'n000149',
 'n000178',
 'guido']